In [ ]:
from tqdm import tqdm
import random
from random import sample
import networkx as nx
import re, logging
import openai, datetime, os

def load_openai_keys():
    keys = []
    with open('../openai_keys.txt', "r") as f:
        for line in f:
            key = line.strip().split()
            keys.append(key[-1])
    return keys
openai_api_keys = load_openai_keys()


def list_equal(a, answers):
    # if set(a) == set(b):
    #     return True
    # else:
    #     return False
    for b in answers:
        if set(a) == set(b):
            return True
    return False

def update_key():
    curr_key = openai_api_keys[0]
    openai.api_key = curr_key
    openai_api_keys.remove(curr_key)
    openai_api_keys.append(curr_key)

def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])

    # Remove any existing handlers
    for handler in logger.handlers:
        logger.removeHandler(handler)
    # Output to file
    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    # # Output to terminal
    # sh = logging.StreamHandler()
    # sh.setFormatter(formatter)
    # logger.addHandler(sh)

    return logger

# multiple-list save to logging
def list2str(l):
    s = ''
    for i in l:
        s += str(i) + '\r'
    return s


In [ ]:
rid = 0
id2rel = dict()
rel2id = dict()
rel2sym = dict()
rel2sym_2 = dict()
relation_txt = ''

infer_rel = list()
with open("../symbolic_tree/1.relations", 'r') as f:
    for line in f:
        _, rel = line.strip().split()

        infer_rel.append(rel)

        relation_txt += rel + ', '
        id2rel[rid] = rel
        rel2id[rel] = rid
        rid += 1

extra_relations = ["greatAuntUncleOf","grandparentOf","greatGrandparentOf","auntUncleOf","siblingOf","secondAuntUncleOf","childOf","grandchildOf","greatGrandchildOf","nieceNephewOf","cousinOf","secondCousinOf","firstCousinOnceRemovedOf", "male", "female"]

for rel in extra_relations:
    id2rel[rid] = rel
    rel2id[rel] = rid
    rid += 1
    
with open("../rel2sym_ri.txt","r") as fr:
    for line in fr:
        rel, sym = line.strip().split()
        rel2sym_2[rel] = sym
        rel2sym[rel] = '$' + sym + '$'


In [ ]:

rh2rules = dict()
id = 1
rule_text = ''
with open('latex_rules.txt','r') as f:
    for line in f:
        rh2rules[rel2sym[infer_rel[id]]] = line.strip().split('\t')[-1]
        rule_text += 'L' + str(id) + ": " + line
        id += 1
# print(rule_text)

In [ ]:
def read_entity(path, eid, id2ent, ent2id ):
    with open(path, 'r') as f:
        for line in f:
            _, ent = line.strip().split()
            if ent not in ent2id:
                id2ent[eid] = ent
                ent2id[ent] = eid
                eid += 1
                
        return eid
    
def get_related_triplets(h, t, G, entpair2rel):
    input_text = ''
    for path in sorted(nx.all_simple_edge_paths(G, h, t, cutoff=5)):
        for edge in path:
            # print(edge)
            if edge in entpair2rel:
                input_text += entpair2rel[edge] + '(' + edge[0] + ',' + edge[1] + ')\n'
                # input_text += edge[0] + ' is the ' + entpair2rel[edge] + ' of ' + edge[1] + '. '
            else:
                input_text += entpair2rel[(edge[1],edge[0])] + '(' + edge[1] + ',' + edge[0] + ')\n'
                # input_text += edge[1] + ' is the ' + entpair2rel[(edge[1],edge[0])] + ' of ' + edge[0] + '. '
    return input_text

def read_all_triplets(path1, path2, id2ent, text):
    triplets = list()
    entpair2rel = dict()
    with open(path1,'r') as f:
        for line in f:
            flag, h, r, t = line.strip().split()
            triplets.append((id2ent[int(h)], id2rel[int(r)], id2ent[int(t)]))
            # entpair2rel[(id2ent[int(h)], id2ent[int(t)])] = rel2sym[id2rel[int(r)]]
            entpair2rel[(id2ent[int(h)], id2ent[int(t)])] = id2rel[int(r)]
            # text += id2ent[int(h)] + ' is the ' + rel2sym[id2rel[int(r)]] + ' of ' + id2ent[int(t)] + '. '
            text += rel2sym[id2rel[int(r)]] + '(' + id2ent[int(h)] + ',' + id2ent[int(t)] + ')\n'
            # text += id2ent[int(t)] + ' is the ' + rel2sym['inverse_' + id2rel[int(r)]] + ' of ' + id2ent[int(h)] + '. '
            # text += id2ent[int(h)] + ' is the ' + id2rel[int(r)] + ' of ' + id2ent[int(t)] + '. '



    with open(path2,'r') as f:
        for line in f:
            flag, h, r, t = line.strip().split()
            if flag == '+':
                # triplets.append((id2ent[int(h)], id2rel[int(r)], id2ent[int(t)]))
                # test_triplets.append((id2ent[int(h)], id2rel[int(r)], id2ent[int(t)]))
                entpair2rel[(id2ent[int(h)], id2ent[int(t)])] = id2rel[int(r)]
                # triplets.append((id2ent[int(h)], id2rel[int(r)], id2ent[int(t)]))
                # text += id2ent[int(h)] + ' is the ' + rel2sym[id2rel[int(r)]] + ' of ' + id2ent[int(t)] + '. '
                text += rel2sym[id2rel[int(r)]] + '(' + id2ent[int(h)] + ',' + id2ent[int(t)] + ')\n'
                # text += id2ent[int(t)] + ' is the ' + rel2sym['inverse_' + id2rel[int(r)]] + ' of ' + id2ent[int(h)] + '. '

                # text += id2ent[int(h)] + ' is the ' + id2rel[int(r)] + ' of ' + id2ent[int(t)] + '. '


    return triplets, entpair2rel, text


def read_class(path, cid, ent2class, id2ent, class_text):
    with open(path, 'r') as f:
        for line in f:
            female, male  = line.strip().split()
            if female == '1':
                # ent2class[id2ent[cid]] = rel2sym['female']
                # ent2class[id2ent[cid]] = 'female'

                # class_text += id2ent[cid] + ' is the ' + rel2sym["female"] + '. '
                class_text += rel2sym['female'] + '(' + id2ent[cid] + ')\n'
                # class_text += id2ent[cid] + ' is a ' + "female" + '. '

                # class_text += ('female'+'(' + id2ent[cid] + ')')
            else:
                # ent2class[id2ent[cid]] = rel2sym['male']
                ent2class[id2ent[cid]] = 'male'

                # class_text += id2ent[cid] + ' is the '+ rel2sym['male'] + '. '
                class_text += rel2sym['male'] + '(' + id2ent[cid] + ')\n'
                # class_text += id2ent[cid] + ' is the '+ 'male' + '. '

                # class_text += ('male'+'(' + id2ent[cid] + ')')

            cid += 1
        return cid, class_text


def read_all_facts(path1, path2, path_class, id2ent, ent2class, cid, text ):
    f_id = 1
    triplets = list()
    test_triplets = list()
    entpair2rel = dict()
    edges = list()
    tri2number = dict()

    with open(path1,'r') as f:
        for line in f:
            flag, h, r, t = line.strip().split()
            triplets.append((id2ent[int(h)], id2rel[int(r)], id2ent[int(t)]))
            edges.append((id2ent[int(h)], id2ent[int(t)]))
            # edges.append((id2ent[int(t)], id2ent[int(h)]))
            entpair2rel[(id2ent[int(h)], id2ent[int(t)])] = rel2sym[id2rel[int(r)]]
            # entpair2rel[(id2ent[int(t)], id2ent[int(h)])] = rel2sym['inverse_' + id2rel[int(r)]]
            # entpair2rel[(id2ent[int(h)], id2ent[int(t)])] = id2rel[int(r)]
            # text += 'F' + str(f_id) + ": " + id2ent[int(h)] + ' is the ' + rel2sym[id2rel[int(r)]] + ' of ' + id2ent[int(t)] + '.\n'
            text += 'F' + str(f_id) + ": " + rel2sym[id2rel[(int(r))]] + '(' + id2ent[int(h)] + ',' + id2ent[int(t)] + ')\n'
            # text += id2ent[int(h)] + ' is the ' + rel2sym[id2rel[int(r)]] + ' of ' + id2ent[int(t)] + '.\n'
            
            tri2number[(id2ent[int(h)], rel2sym[id2rel[int(r)]], id2ent[int(t)])] = 'F' + str(f_id)
            f_id += 1
            # text += 'F' + str(f_id) + ": " + id2ent[int(t)] + ' is the ' + rel2sym['inverse_' + id2rel[int(r)]] + ' of ' + id2ent[int(h)] + '.\n'
            # # text += id2ent[int(t)] + ' is the ' + rel2sym['inverse_' + id2rel[int(r)]] + ' of ' + id2ent[int(h)] + '.\n'
            
            # tri2number[(id2ent[int(t)], rel2sym['inverse_' + id2rel[int(r)]], id2ent[int(h)])] = 'F' + str(f_id)
            
            # f_id += 1
            
            # text += id2ent[int(h)] + ' is the ' + id2rel[int(r)] + ' of ' + id2ent[int(t)] + '. '

    with open(path2,'r') as f:
        for line in f:
            flag, h, r, t = line.strip().split()
            if flag == '+':
                # triplets.append((id2ent[int(h)], id2rel[int(r)], id2ent[int(t)]))
                test_triplets.append((id2ent[int(h)], rel2sym[id2rel[int(r)]], id2ent[int(t)]))
                # entpair2rel[(id2ent[int(h)], id2ent[int(t)])] = id2rel[int(r)]
                
                triplets.append((id2ent[int(h)], id2rel[int(r)], id2ent[int(t)]))
                # text += 'F' + str(f_id) + ": " + id2ent[int(h)] + ' is the ' + rel2sym[id2rel[int(r)]] + ' of ' + id2ent[int(t)] + '. '
                # f_id += 1
                
                # text += id2ent[int(t)] + ' is the ' + rel2sym['inverse_' + id2rel[int(r)]] + ' of ' + id2ent[int(h)] + '. '

                # text += id2ent[int(h)] + ' is the ' + id2rel[int(r)] + ' of ' + id2ent[int(t)] + '. '

    with open(path_class, 'r') as f:
        for line in f:
            female, male  = line.strip().split()
            if female == '1':
                ent2class[id2ent[cid]] = rel2sym['female']
                # ent2class[id2ent[cid]] = 'female'
        
                # text += 'F' + str(f_id) + ": " + id2ent[cid] + ' is the ' + rel2sym["female"] + '.\n'
                text += 'F' + str(f_id) + ': ' + rel2sym['female'] + '(' + id2ent[cid] + ')\n'
                # text += id2ent[cid] + ' is the ' + rel2sym["female"] + '.\n'

                tri2number[(id2ent[cid], 'gender', rel2sym['female'])] = 'F' + str(f_id)
                # class_text += id2ent[cid] + ' is a ' + "female" + '. '

                # class_text += ('female'+'(' + id2ent[cid] + ')')
            else:
                ent2class[id2ent[cid]] = rel2sym['male']
                # ent2class[id2ent[cid]] = 'male'

                # text += 'F' + str(f_id) + ": " + id2ent[cid] + ' is the '+ rel2sym['male'] + '.\n'
                text += 'F' + str(f_id) + ': ' + rel2sym['male'] + '(' + id2ent[cid] + ')\n'

                # text += id2ent[cid] + ' is the '+ rel2sym['male'] + '.\n'

                tri2number[(id2ent[cid], 'gender', rel2sym['male'])] = 'F' + str(f_id)
                
                # class_text += id2ent[cid] + ' is the '+ 'male' + '. '

                # class_text += ('male'+'(' + id2ent[cid] + ')')
            f_id += 1
            cid += 1
    return triplets, test_triplets, entpair2rel, cid, text, edges, tri2number
    
def get_explain_grounding_truth(test_triplets, edges, entpair2rel, ent2class, rel2rules,  tri2number, rule2number):

    # Define the logical rules that the paths should match
    def logical_rules(entpair2rel, path, rule):
        path_number = list()
        for i in range(len(path)-1):
            if (path[i], path[i+1]) in entpair2rel:
                if entpair2rel[(path[i], path[i+1])] == rule[i]:
                    path_number.append(tri2number[(path[i], entpair2rel[(path[i], path[i+1])], path[i+1])])
            elif '$r45$' == rule[i]:
                path_number.append(tri2number[(path[i+1], entpair2rel[(path[i+1], path[i])], path[i])])

            else:
                
                return None
        return path_number



    # Define your knowledge graph using the NetworkX library
    G = nx.Graph()
    G.add_edges_from(edges)

    fact2explain = dict()
    fact2rule = dict()
    for tri in test_triplets:
        h = tri[0]
        r = tri[1]
        t = tri[2]
        rule = rel2rules[r]
        length = len(rule)
        
        all_paths = list()
        # Find all paths that match the logical rules using NetworkX's all_simple_paths() function
        for path in nx.all_simple_paths(G, source=h, target=t, cutoff=length):
            # print("path", path)
            path_number = logical_rules(entpair2rel, path, rule)
            if path_number:
                if ent2class[h] == rule[-1]:
                    path_number.append(tri2number[(h,'gender',ent2class[h])])
                    
                    # path_number.append(rule2number[r])
                    all_paths.append(path_number)
        fact2explain[(h, r, t)] = all_paths
        # fact2explain[(h,r,t)] = [rule2number[r]]
        fact2rule[(h,r,t)] = rule2number[r]

    return fact2explain, fact2rule


def read_rules(path, rel2sym):
    rel2rules = dict()
    rule2number = dict()
    l_id = 1
    with open(path, 'r') as f:
        for line in f:
            lst = line.strip().split('\t')
            # replace symbol 
            new_lst = list()
            for l in lst:
                new_lst.append(rel2sym[l])
            
            rel2rules[new_lst[0]] = new_lst[1:]
            rule2number[new_lst[0]] = 'L' + str(l_id)
            l_id += 1
    return rel2rules, rule2number




In [ ]:
# from rule_parent.txt get the length of each rule
rule_length = dict()
with open('rule_tab.txt', 'r') as f:
    for line in f:
        lst = line.strip().split('\t')
        rule_length[rel2sym[lst[0]]] = len(lst[1:]) 

In [ ]:
# read data
for i in range(0, 1):
    
    eid = 0
    cid = 0
    id2ent = dict()
    ent2id = dict()
    ent2class = dict()
    ent2triplets = dict()
    class_text = ''
    text = ''

    path_ent = "../symbolic_tree/" + str(i) + ".individuals"
    path_rel1 = "../symbolic_tree/"+str(i)+".relations.data"
    path_rel2 = "../symbolic_tree/"+str(i)+".relations.data.inf"
    path_class = "../symbolic_tree/" + str(i) + ".classes.data"

    path_rule = 'rule_tab.txt'

    eid = read_entity(path_ent,eid, id2ent,ent2id)
    # cid, class_text = read_class(path_class, cid, ent2class, id2ent, class_text)
    
    # triplets, entpair2rel, text = read_all_triplets(path_rel1, path_rel2, id2ent, text)

    
    triplets, test_triplets, entpair2rel, cid, text, edges, tri2number = read_all_facts(path_rel1, path_rel2, path_class, id2ent, ent2class, cid, text)
    
    rel2rules, rule2number = read_rules(path_rule, rel2sym)
    fact2explain, fact2rule= get_explain_grounding_truth(test_triplets, edges, entpair2rel, ent2class, rel2rules, tri2number, rule2number)
    # test_questions = random.sample(triplets, int(len(triplets) * 0.2))
    # train_questions = triplets.copy()
    # for t in test_questions:
    #     train_questions.remove(t)

In [ ]:
fact2explain

In [ ]:
fact2explain[test_triplets[0]]

In [ ]:
def dict2str(d):
    s = ''
    for k in d:
        s += str(k) + '\t' + str(d[k]) + '\r'
    return s

In [ ]:
nowTime = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
dir = 'logs/first_order_standard/'
if not os.path.exists(dir):
        os.makedirs(dir)
logger = get_logger(os.path.join(dir, nowTime + '.log'), verbosity=1)

model = "gpt-3.5-turbo"
# model = "gpt-4"
logging.info('model: ' + model)

record_flag = False

true_num = 0
false_num = 0
num = 0



for triple in tqdm(test_triplets):
    h, r, t = triple
    # text_pred = h + ' is the ' + r + ' of ' + t + '.'
    text_pred = r + '(' + h + ', ' + t + ')'
    # text_pred = 'Is ' + h + ' the ' + r + ' of ' + t + '? \n '
    # input_rule_text = get_rule_text(rule_heads[r], rule2text)
    # input_text = get_related_triplets(h, t, G, entpair2rel)
    # input_class_text = h + " is the " + ent2class[h] + '. ' 

    # print(input_text)
    # print("We also have some facts. " + class_text + input_text + text_pred + ' If yes, please answer only with 1 else 0')
    
    # print(text_pred)
   
    
    server_error_cnt = 0
    
    while server_error_cnt<10:
        try:
            message = {
                        'system': "You are a helpful assistant. I will give you some logical rules and facts. Please select some facts to explain the following statement. ",
                        'user': "I will give you some logical rules and facts.\nThe facts are:\n" + text + "\nThe logical rule is:\n" + rh2rules[r] + "\nPlease select " + str(rule_length[r]) + " facts that matches the logical rule. The selected facts and the given rule can entail the following statement.\nThe statement is: " + text_pred + "\nThe selected " + str(rule_length[r]) +  " facts are: ",
                        }
            update_key()
            response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            # model="gpt-4",

            messages=[
                 
                    {"role": "system", "content": message['system']},
                    {"role": "user", "content": message['user']},
                                
                    # {"role": "system", "content": "You are a helpful assistant. "},

                    # {"role": "system", "content": "You are a helpful assistant. I will give you some logical rules and facts. You need select " + str(rule_length[r]) + " facts to explain the following statement. "},
                    # {"role": "system", "content": "You are a helpful assistant. I will give you some logical rules and facts. Please select some facts to explain the following statement. "},
                    
                    # {"role": "user", "content": "We have some logical rules. " + rules_text + "We also have some facts. " + class_text + text + text_pred + ' If yes, please answer only with 1 else 0'},
                    # {"role": "user", "content": "We have some logical rules.\n" + input_rule_text + "We also have some facts.\n" + input_text + text_pred + 'If yes, please answer only with 1 else 0'},
                    # {"role": "user", "content": "We have some logical rules.\n" + rules_text + "We also have some facts.\n" + input_class_text + input_text + text_pred + 'If yes, please answer only with 1 else 0'},
                    # {"role": "user", "content": " I will give you some logical rules and facts. You only need select one logical rule to explain the following statement. The logical rules are:\n" + rule_text + "\nThe facts are: " + text + "\nThe statement is: " + text_pred + "\nYou can find the logical rule that entail the statement and then grounding the logical rule to select related facts. Please select the numbers of one logical rule and multiple facts. The number list is: "},
                    # {"role": "user", "content": " I will give you some logical rules and facts.\nThe facts are:\n" + text + "\nThe logical rule is:\n" + rh2rules[r] + "\nPlease select " + str(rule_length[r]) + " facts. The selected facts and the given rule can entail the following statement. \nThe statement is: " + text_pred + "\nThe selected " + str(rule_length[r]) +  " facts are: "},
                    # {"role": "user", "content": " I will give you some logical rules and facts.\nThe facts are:\n" + text + "\nThe logical rule is:\n" + rh2rules[r] + "\nPlease select " + str(rule_length[r]) + " facts that matches the logical rule. The selected facts and the given rule can entail the following statement.\nThe statement is: " + text_pred + "\nThe selected " + str(rule_length[r]) +  " facts are: "},
                    # {"role": "user", "content": " I will give you some logical rules and facts.\nThe facts are:\n" + text + "\nThe logical rule is:\n" + rh2rules[r] + "\nPlease select multiple facts that matches the logical rule. The selected facts and the given rule can entail the following statement.\nThe statement is: " + text_pred + "\nThe selected facts are: "},
                    # {"role": "user", "content": "I will give you some logical rules and facts.\nThe facts are:\n" + text + "\nThe logical rule is:\n" + rh2rules[r] + "\nPlease select " + str(rule_length[r]) + " facts that matches the logical rule. The selected facts and the given rule can entail the following statement.\nThe statement is: " + text_pred + "\nThe selected " + str(rule_length[r]) +  " facts are: "},
                    # {"role": "user", "content": " I will give you some logical rules and facts.\nThe facts are:\n" + text + "\nThe logical rule is:\n" + rh2rules[r] + "\nPlease select multiple facts that matches the logical rule. The selected facts and the given rule can entail the following statement.\nThe statement is: " + text_pred + "\nThe selected facts are: "},
                    
                ],
            temperature=0,

            )
            
            results = response['choices'][0]['message']['content']
            # print("I will give you some logical rules and facts.\nThe facts are:\n" + text + "\nThe logical rule is:\n" + rh2rules[r] + "\nPlease select " + str(rule_length[r]) + " facts that matches the logical rule. The selected facts and the given rule can entail the following statement.\nThe statement is: " + text_pred + "\nThe selected " + str(rule_length[r]) +  " facts are: ")
            if record_flag == False:

                logger.info('message: \n' + dict2str(message)) 

                record_flag = True
            
            # print(results)
            num += 1
            number_list = re.findall(r'[FL]\d+', results)
            logger.info('LLM: ' + (', '.join(number_list)) + '\t' + "grounding_truth: " + (', '.join(fact2explain[(h,r,t)][0])) + '\t' + "results: " + results)
            # print('LLM: ', number_list)
            # print("grounding_truth: ", fact2explain[(h,r,t)])
            if list_equal(number_list, fact2explain[(h,r,t)]):
                print("correct")
                true_num += 1
            else:
                false_num += 1
            break
        except Exception as e:
            server_error_cnt += 1
            print(e)

print("accuracy: ", true_num/num)

In [ ]:
fact2explain[('Lorenz', '$r2$', 'Lea')]

In [ ]:
list_equal(['F1', 'F1', 'F3'], ['F1', 'F3'])

## for each triplet to find grounded rules

In [ ]:
for triplet in triplets:
    rel = triplet[1]

    

In [ ]:
import networkx as nx
# Define your knowledge graph using the NetworkX library
G = nx.Graph()
G.add_edges_from([(1,2), (2,3), (3,4), (4,5), (5,6), (6,7), (7,8)])

# Define the logical rules that the paths should match
def logical_rules(path):
    return path[0] == 1 and path[-1] == 8 and len(path) >= 4

# Find all paths that match the logical rules using NetworkX's all_simple_paths() function
all_paths = []
for path in nx.all_simple_paths(G, source=1, target=8):
    print("path", path)
    if logical_rules(path):
        all_paths.append(path)

# Print the resulting paths
print(all_paths)

# add inverse relation into KG

In [ ]:
# add inverse relation for each relation
rel2sym = dict()
sym2rel = dict()
import random
from random import sample

with open('rel2sym.txt', 'r') as f:
    for line in f:
        line = line.strip()
        rel, sym = line.split('\t')
        rel2sym[rel] = sym
        sym2rel[sym] = rel


for t in list(rel2sym.keys()):
    t_prime = 'inverse_' + t
    # random sample a symbol
    sym = ''.join(sample('abcdefghijklmnopqrstuvwxyz',random.randint(4,8)))
    rel2sym[t_prime] = sym
    sym2rel[sym] = t_prime

# write rel2sym_inverse.txt to file
with open('rel2sym_inverse.txt', 'w') as f:
    for t in rel2sym:
        f.write(t + '\t' + rel2sym[t] + '\n')

In [ ]:
rel2sym.keys()

# tranlate the logical rule as latex format and save to file "rule_latex.txt"

In [ ]:
#  tranlate the logical rule as latex format and save to file "rule_latex.txt"
# input: sisterOf	inverse_parentOf	parentOf	female
# output: $inverse_parentOf(a,b) \land parentOf(b,c) \land female(a) \rightarrow sisterOf(a,c) $
with open("rule_latex.txt", 'w') as fw:
    with open("rule_parents.txt", 'r') as f:
        for line in f:
            lst = line.strip().split('\t')
            fw.writelines('$')
            h = ord('A')
            for rel in lst[1:-1]:
                t = h + 1
                fw.writelines(rel2sym_2[rel] + '(' + chr(h) + ',' + chr(t) + ') \land ')
                h = t
            fw.writelines(rel2sym_2[lst[-1]] + '(A) \\rightarrow ' + rel2sym_2[lst[0]] + '(A,' + chr(t) + ')$\n')

In [ ]:
# tranlate the logical rule as natural language and save to file "rule_latex.txt"
with open("rule_language.txt", 'w') as fw:
    with open("rule_parents.txt", 'r') as f:
        for line in f:
            lst = line.strip().split('\t')
            fw.writelines('If ')
            h = ord('A')
            for rel in lst[1:-1]:
                t = h + 1
                fw.writelines(chr(h) + ' is the $' + rel2sym_2[rel] + '$ of ' + chr(t) + ' and ')
                h = t
            fw.writelines('A is the $' + rel2sym_2[lst[-1]] + '$, then A is the $' + rel2sym_2[lst[0]] + '$ of ' + chr(t) + '.\n')

In [ ]:
template = dict()

for key in rule_length:
    template[key] = 'If '
    ent_h = ord('A')
    for i in range(rule_length[key]):
        ent_h = ent_h
        ent_t = ent_h + 1
        template[key] += chr(ent_h) + ' is the ## of ' + chr(ent_t) + ' and '
        # template[key] += '##(' + chr(ent_h) + ',' + chr(ent_t) + ') \land '
        ent_h = ent_t 
    template[key] += 'A is the ++, then A is the ' + key + ' of ' + chr(ent_t) + '.'

# translate the family relation into symbolic and save a new file.txt

In [ ]:
with open("rule_symbolic_tab.txt", 'w') as fw:
    with open("rule_parents.txt", 'r') as f:
        for line in f:
            lsts = line.strip().split('\t')
            for lst in lsts[:-1]:
                fw.writelines(rel2sym[lst] + '\t')
            fw.writelines(rel2sym[lsts[-1]] + '\n')
            

In [ ]:
# 正则表达式 F或L+数字 
import re
a = 'F5,L1,F45'
b = re.findall(r'[FL]\d+', a)
print(b)

In [ ]:
# list转换成集合
a = ['F1', 'F1', 'F3']
b = set(a)
c = set(['F1', 'F3'])
# 判断两个集合是否相等

b == c


In [ ]:
a = [1,2,3]
b = [3,2,1]
list_equal(a,b)